<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [2]:
from nba_api.stats.static import teams
from nba_api.stats.library.parameters import GameDate, SeasonID, SeasonNullable
import functools
import numpy as np
import pandas as pd
from datetime import datetime
from nba_api.stats.endpoints import leaguegamefinder, boxscoresummaryv2, teaminfocommon, leaguegamelog
from nba_api.stats.static import players
from src import data_collection as dc

import requests
import json

In [125]:
df = pd.read_csv('data/processed/seasons/1982-83.csv', dtype={'GAME_ID':str})
df.head()

,GAME_ID,SEASON_ID,GAME_DATE,TEAM_ID_1,TEAM_ID_2,TEAM_ABBREVIATION_1,TEAM_ABBREVIATION_2,TEAM_NAME_1,TEAM_NAME_2,MATCHUP_1,...,TOV_1,TOV_2,PF_1,PF_2,PTS_1,PTS_2,PLUS_MINUS_1,PLUS_MINUS_2,VIDEO_AVAILABLE_1,VIDEO_AVAILABLE_2
0,0028200001,21982,1982-10-29,1610612741,1610612751,CHI,NJN,Chicago Bulls,New Jersey Nets,CHI @ NJN,...,NaN,NaN,29.0,28.0,102,114,-12,12,0,0
1,0028200002,21982,1982-10-29,1610612757,1610612758,POR,KCK,Portland Trail Blazers,Kansas City Kings,POR @ KCK,...,NaN,NaN,27.0,36.0,111,113,-2,2,0,0
2,0028200003,21982,1982-10-29,1610612754,1610612764,IND,WAS,Indiana Pacers,Washington Bullets,IND vs. WAS,...,NaN,NaN,28.0,21.0,93,90,3,-3,0,0
3,0028200004,21982,1982-10-29,1610612746,1610612756,SDC,PHX,San Diego Clippers,Phoenix Suns,SDC vs. PHX,...,NaN,NaN,NaN,NaN,99,113,-14,14,0,0
4,0028200005,21982,1982-10-29,1610612745,1610612760,HOU,SEA,Houston Rockets,Seattle SuperSonics,HOU @ SEA,...,NaN,NaN,26.0,25.0,95,128,-33,33,0,0


In [148]:
game_ids = df['GAME_ID'].values
huh = pd.DataFrame()
i = 0
for game in game_ids:
    i += 1
    derp = boxscoresummaryv2.BoxScoreSummaryV2(game).get_data_frames()
    row = pd.concat((functools.reduce(lambda x,y: pd.merge(x,y, on='GAME_ID', how='outer'), \
                                      derp[-3:] + [derp[0]]), derp[4]), 1)
    huh = pd.concat((huh, row))
    if i == 10:
        break


In [150]:
huh.T

,0,0,0,0,0,0,0,0,0,0
GAME_ID,0028200001,0028200002,0028200003,0028200004,0028200005,0028200006,0028200007,0028200008,0028200009,0028200010
LAST_GAME_ID,0028100675,0028100821,0028100866,0028100904,0048100010,0028100867,0028100784,0028100853,0028100941,0028100931
LAST_GAME_DATE_EST,1982-03-05T00:00:00,1982-03-30T00:00:00,1982-04-07T00:00:00,1982-04-13T00:00:00,1982-04-25T00:00:00,1982-04-07T00:00:00,1982-03-24T00:00:00,1982-04-04T00:00:00,1982-04-18T00:00:00,1982-04-16T00:00:00
LAST_GAME_HOME_TEAM_ID,1610612751,1610612758,1610612754,1610612746,1610612760,1610612765,1610612739,1610612752,1610612762,1610612747
LAST_GAME_HOME_TEAM_CITY,New Jersey,Kansas City,Indiana,San Diego,Seattle,Detroit,Cleveland,New York,Utah,Los Angeles
LAST_GAME_HOME_TEAM_NAME,Nets,Kings,Pacers,Clippers,SuperSonics,Pistons,Cavaliers,Knicks,Jazz,Lakers
LAST_GAME_HOME_TEAM_ABBREVIATION,NJN,KCK,IND,SDC,SEA,DET,CLE,NYK,UTH,LAL
LAST_GAME_HOME_TEAM_POINTS,107,99,85,94,104,120,115,106,128,125
LAST_GAME_VISITOR_TEAM_ID,1610612741,1610612757,1610612764,1610612756,1610612745,1610612737,1610612738,1610612755,1610612759,1610612744
LAST_GAME_VISITOR_TEAM_CITY,Chicago,Portland,Washington,Phoenix,Houston,Atlanta,Boston,Philadelphia,San Antonio,Golden State


In [142]:
derp[-3:] + [derp[0]]

[      GAME_ID LAST_GAME_ID   LAST_GAME_DATE_EST  LAST_GAME_HOME_TEAM_ID  \
 0  0028200001   0028100675  1982-03-05T00:00:00              1610612751   
 
   LAST_GAME_HOME_TEAM_CITY LAST_GAME_HOME_TEAM_NAME  \
 0               New Jersey                     Nets   
 
   LAST_GAME_HOME_TEAM_ABBREVIATION  LAST_GAME_HOME_TEAM_POINTS  \
 0                              NJN                         107   
 
    LAST_GAME_VISITOR_TEAM_ID LAST_GAME_VISITOR_TEAM_CITY  \
 0                 1610612741                     Chicago   
 
   LAST_GAME_VISITOR_TEAM_NAME LAST_GAME_VISITOR_TEAM_CITY1  \
 0                       Bulls                          CHI   
 
    LAST_GAME_VISITOR_TEAM_POINTS  
 0                             90  ,
       GAME_ID  HOME_TEAM_ID  VISITOR_TEAM_ID        GAME_DATE_EST  \
 0  0028200001    1610612751       1610612741  1982-10-29T00:00:00   
 
    HOME_TEAM_WINS  HOME_TEAM_LOSSES SERIES_LEADER  
 0               4                 2    New Jersey  ,
       GAME_ID  VIDEO_

,GAME_ID,LAST_GAME_ID,LAST_GAME_DATE_EST,LAST_GAME_HOME_TEAM_ID,LAST_GAME_HOME_TEAM_CITY,LAST_GAME_HOME_TEAM_NAME,LAST_GAME_HOME_TEAM_ABBREVIATION,LAST_GAME_HOME_TEAM_POINTS,LAST_GAME_VISITOR_TEAM_ID,LAST_GAME_VISITOR_TEAM_CITY,...,VISITOR_TEAM_ID_y,SEASON,LIVE_PERIOD,LIVE_PC_TIME,NATL_TV_BROADCASTER_ABBREVIATION,LIVE_PERIOD_TIME_BCAST,WH_STATUS_y,GAME_DATE,ATTENDANCE,GAME_TIME
0,0028200001,0028100675,1982-03-05T00:00:00,1610612751,New Jersey,Nets,NJN,107,1610612741,Chicago,...,1610612741,1982,4,None,None,Q4 -,1,"FRIDAY, OCTOBER 29, 1982",11042,


In [131]:
derp

[         GAME_DATE_EST GAME_SEQUENCE     GAME_ID  GAME_STATUS_ID  \
 0  1982-10-29T00:00:00          None  0028200001               3   
 
   GAME_STATUS_TEXT         GAMECODE  HOME_TEAM_ID  VISITOR_TEAM_ID SEASON  \
 0                   19821029/CHINJN    1610612751       1610612741   1982   
 
    LIVE_PERIOD LIVE_PC_TIME NATL_TV_BROADCASTER_ABBREVIATION  \
 0            4         None                             None   
 
   LIVE_PERIOD_TIME_BCAST  WH_STATUS  
 0                 Q4  -           1  ,
 Empty DataFrame
 Columns: [LEAGUE_ID, TEAM_ID, TEAM_ABBREVIATION, TEAM_CITY, PTS_PAINT, PTS_2ND_CHANCE, PTS_FB, LARGEST_LEAD, LEAD_CHANGES, TIMES_TIED, TEAM_TURNOVERS, TOTAL_TURNOVERS, TEAM_REBOUNDS, PTS_OFF_TO]
 Index: [],
 Empty DataFrame
 Columns: [OFFICIAL_ID, FIRST_NAME, LAST_NAME, JERSEY_NUM]
 Index: [],
 Empty DataFrame
 Columns: [PLAYER_ID, FIRST_NAME, LAST_NAME, JERSEY_NUM, TEAM_ID, TEAM_CITY, TEAM_NAME, TEAM_ABBREVIATION]
 Index: [],
                   GAME_DATE  ATTENDANCE G

In [40]:
team_ids = teams['id']
for team_id in [1610612738]:
    df = leaguegamefinder.LeagueGameFinder(team_id_nullable=team_id, season_nullable="1975-76").get_data_frames()[0]
    break

In [4]:
derp = pd.read_csv("data/raw/seasons/1946-47.csv")
derp.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,21946,1610612752,NYK,New York Knicks,24600001,1946-11-01,NYK @ HUS,W,0,24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68,2,0
1,21946,1610610035,HUS,Toronto Huskies,24600001,1946-11-01,HUS vs. NYK,L,0,25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66,-2,0
2,21946,1610610032,PRO,Providence Steamrollers,24600002,1946-11-02,PRO vs. BOS,W,0,21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59,6,0
3,21946,1610612738,BOS,Boston Celtics,24600002,1946-11-02,BOS @ PRO,L,0,21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53,-6,0
4,21946,1610612752,NYK,New York Knicks,24600004,1946-11-02,NYK @ CHS,L,0,16,...,NaN,NaN,NaN,NaN,NaN,NaN,22.0,47,-16,0


In [7]:
derp = derp.groupby(['GAME_ID', 'SEASON_ID', 'GAME_DATE']).agg(list)
derp

,,,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,FG3M,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
GAME_ID,SEASON_ID,GAME_DATE,,,,,,,,,,,,,,,,,,,,,
24600001,21946,1946-11-01,"[[1610612752, 1610610035]]","[[NYK, HUS]]","[[New York Knicks, Toronto Huskies]]","[[NYK @ HUS, HUS vs. NYK]]","[[W, L]]","[[0, 0]]","[[24, 25]]","[[nan, nan]]","[[nan, nan]]","[[nan, nan]]",...,"[[nan, nan]]","[[nan, nan]]","[[nan, nan]]","[[nan, nan]]","[[nan, nan]]","[[nan, nan]]","[[nan, nan]]","[[68, 66]]","[[2, -2]]","[[0, 0]]"
24600002,21946,1946-11-02,"[[1610610032, 1610612738]]","[[PRO, BOS]]","[[Providence Steamrollers, Boston Celtics]]","[[PRO vs. BOS, BOS @ PRO]]","[[W, L]]","[[0, 0]]","[[21, 21]]","[[nan, nan]]","[[nan, nan]]","[[nan, nan]]",...,"[[nan, nan]]","[[nan, nan]]","[[nan, nan]]","[[nan, nan]]","[[nan, nan]]","[[nan, nan]]","[[nan, nan]]","[[59, 53]]","[[6, -6]]","[[0, 0]]"
24600003,21946,1946-11-02,"[[1610610031, 1610610034]]","[[PIT, BOM]]","[[Pittsburgh Ironmen, St. Louis Bombers]]","[[PIT @ BOM, BOM vs. PIT]]","[[L, W]]","[[0, 0]]","[[16, 20]]","[[72.0, 59.0]]","[[0.222, 0.339]]","[[nan, nan]]",...,"[[nan, nan]]","[[nan, nan]]","[[nan, nan]]","[[nan, nan]]","[[nan, nan]]","[[nan, nan]]","[[25.0, 21.0]]","[[51, 56]]","[[-5, 5]]","[[0, 0]]"
24600004,21946,1946-11-02,"[[1610612752, 1610610025]]","[[NYK, CHS]]","[[New York Knicks, Chicago Stags]]","[[NYK @ CHS, CHS vs. NYK]]","[[L, W]]","[[0, 0]]","[[16, 21]]","[[nan, nan]]","[[nan, nan]]","[[nan, nan]]",...,"[[nan, nan]]","[[nan, nan]]","[[nan, nan]]","[[nan, nan]]","[[nan, nan]]","[[nan, nan]]","[[22.0, 20.0]]","[[47, 63]]","[[-16, 16]]","[[0, 0]]"
24600005,21946,1946-11-02,"[[1610610028, 1610610036]]","[[DEF, WAS]]","[[Detroit Falcons, Washington Capitols]]","[[DEF vs. WAS, WAS @ DEF]]","[[L, W]]","[[0, 0]]","[[10, 18]]","[[nan, nan]]","[[nan, nan]]","[[nan, nan]]",...,"[[nan, nan]]","[[nan, nan]]","[[nan, nan]]","[[nan, nan]]","[[nan, nan]]","[[nan, nan]]","[[nan, nan]]","[[33, 50]]","[[-17, 17]]","[[0, 0]]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24600327,21946,1947-03-29,"[[1610610028, 1610610035]]","[[DEF, HUS]]","[[Detroit Falcons, Toronto Huskies]]","[[DEF vs. HUS, HUS @ DEF]]","[[W, L]]","[[0, 0]]","[[24, 25]]","[[nan, nan]]","[[nan, nan]]","[[nan, nan]]",...,"[[nan, nan]]","[[nan, nan]]","[[nan, nan]]","[[nan, nan]]","[[nan, nan]]","[[nan, nan]]","[[20.0, 21.0]]","[[66, 63]]","[[3, -3]]","[[0, 0]]"
24600328,21946,1947-03-29,"[[1610612738, 1610610034]]","[[BOS, BOM]]","[[Boston Celtics, St. Louis Bombers]]","[[BOS @ BOM, BOM vs. BOS]]","[[L, W]]","[[0, 0]]","[[24, 23]]","[[93.0, 90.0]]","[[0.258, 0.256]]","[[nan, nan]]",...,"[[nan, nan]]","[[nan, nan]]","[[nan, nan]]","[[nan, nan]]","[[nan, nan]]","[[nan, nan]]","[[17.0, 13.0]]","[[55, 59]]","[[-4, 4]]","[[0, 0]]"
24600329,21946,1947-03-30,"[[1610612752, 1610612744]]","[[NYK, PHW]]","[[New York Knicks, Philadelphia Warriors]]","[[NYK @ PHW, PHW vs. NYK]]","[[L, W]]","[[0, 0]]","[[25, 27]]","[[nan, nan]]","[[nan, nan]]","[[nan, nan]]",...,"[[nan, nan]]","[[nan, nan]]","[[nan, nan]]","[[nan, nan]]","[[nan, nan]]","[[nan, nan]]","[[nan, nan]]","[[72, 76]]","[[-4, 4]]","[[0, 0]]"


In [105]:
string = 'derp'
string.replace('er', 'pu')

'dpup'

In [104]:
columns_new = sum([[x + '_1'] + [x + '_2'] for x in derp.columns], [])
columns_old = derp.columns
herp = derp.apply(pd.Series.explode).reset_index()
pd.concat([herp.iloc[:, 0:3]] + [pd.DataFrame(herp[x].to_list(), \
                        columns = [columns_new[i*2], columns_new[i*2+1]]) for i, x in enumerate(columns_old)], 1)

,GAME_ID,SEASON_ID,GAME_DATE,TEAM_ID_1,TEAM_ID_2,TEAM_ABBREVIATION_1,TEAM_ABBREVIATION_2,TEAM_NAME_1,TEAM_NAME_2,MATCHUP_1,...,TOV_1,TOV_2,PF_1,PF_2,PTS_1,PTS_2,PLUS_MINUS_1,PLUS_MINUS_2,VIDEO_AVAILABLE_1,VIDEO_AVAILABLE_2
0,24600001,21946,1946-11-01,1610612752,1610610035,NYK,HUS,New York Knicks,Toronto Huskies,NYK @ HUS,...,NaN,NaN,NaN,NaN,68,66,2,-2,0,0
1,24600002,21946,1946-11-02,1610610032,1610612738,PRO,BOS,Providence Steamrollers,Boston Celtics,PRO vs. BOS,...,NaN,NaN,NaN,NaN,59,53,6,-6,0,0
2,24600003,21946,1946-11-02,1610610031,1610610034,PIT,BOM,Pittsburgh Ironmen,St. Louis Bombers,PIT @ BOM,...,NaN,NaN,25.0,21.0,51,56,-5,5,0,0
3,24600004,21946,1946-11-02,1610612752,1610610025,NYK,CHS,New York Knicks,Chicago Stags,NYK @ CHS,...,NaN,NaN,22.0,20.0,47,63,-16,16,0,0
4,24600005,21946,1946-11-02,1610610028,1610610036,DEF,WAS,Detroit Falcons,Washington Capitols,DEF vs. WAS,...,NaN,NaN,NaN,NaN,33,50,-17,17,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326,24600327,21946,1947-03-29,1610610028,1610610035,DEF,HUS,Detroit Falcons,Toronto Huskies,DEF vs. HUS,...,NaN,NaN,20.0,21.0,66,63,3,-3,0,0
327,24600328,21946,1947-03-29,1610612738,1610610034,BOS,BOM,Boston Celtics,St. Louis Bombers,BOS @ BOM,...,NaN,NaN,17.0,13.0,55,59,-4,4,0,0
328,24600329,21946,1947-03-30,1610612752,1610612744,NYK,PHW,New York Knicks,Philadelphia Warriors,NYK @ PHW,...,NaN,NaN,NaN,NaN,72,76,-4,4,0,0
329,24600330,21946,1947-03-30,1610610026,1610612738,CLR,BOS,Cleveland Rebels,Boston Celtics,CLR vs. BOS,...,NaN,NaN,NaN,NaN,66,71,-5,5,0,0


#### pd.DataFrame(herp['TEAM_ID'].to_list(),  columns = [columns[0*2], columns[0*2+1]])

In [102]:
string = 'data/raw/teams.csv'
string.split('/')[-1]

'teams.csv'

In [56]:
herp = derp.apply(pd.Series.explode)
herp

,,,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,FG3M,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
GAME_ID,SEASON_ID,GAME_DATE,,,,,,,,,,,,,,,,,,,,,
24600001,21946,1946-11-01,"[1610612752, 1610610035]","[NYK, HUS]","[New York Knicks, Toronto Huskies]","[NYK @ HUS, HUS vs. NYK]","[W, L]","[0, 0]","[24, 25]","[nan, nan]","[nan, nan]","[nan, nan]",...,"[nan, nan]","[nan, nan]","[nan, nan]","[nan, nan]","[nan, nan]","[nan, nan]","[nan, nan]","[68, 66]","[2, -2]","[0, 0]"
24600002,21946,1946-11-02,"[1610610032, 1610612738]","[PRO, BOS]","[Providence Steamrollers, Boston Celtics]","[PRO vs. BOS, BOS @ PRO]","[W, L]","[0, 0]","[21, 21]","[nan, nan]","[nan, nan]","[nan, nan]",...,"[nan, nan]","[nan, nan]","[nan, nan]","[nan, nan]","[nan, nan]","[nan, nan]","[nan, nan]","[59, 53]","[6, -6]","[0, 0]"
24600003,21946,1946-11-02,"[1610610031, 1610610034]","[PIT, BOM]","[Pittsburgh Ironmen, St. Louis Bombers]","[PIT @ BOM, BOM vs. PIT]","[L, W]","[0, 0]","[16, 20]","[72.0, 59.0]","[0.222, 0.339]","[nan, nan]",...,"[nan, nan]","[nan, nan]","[nan, nan]","[nan, nan]","[nan, nan]","[nan, nan]","[25.0, 21.0]","[51, 56]","[-5, 5]","[0, 0]"
24600004,21946,1946-11-02,"[1610612752, 1610610025]","[NYK, CHS]","[New York Knicks, Chicago Stags]","[NYK @ CHS, CHS vs. NYK]","[L, W]","[0, 0]","[16, 21]","[nan, nan]","[nan, nan]","[nan, nan]",...,"[nan, nan]","[nan, nan]","[nan, nan]","[nan, nan]","[nan, nan]","[nan, nan]","[22.0, 20.0]","[47, 63]","[-16, 16]","[0, 0]"
24600005,21946,1946-11-02,"[1610610028, 1610610036]","[DEF, WAS]","[Detroit Falcons, Washington Capitols]","[DEF vs. WAS, WAS @ DEF]","[L, W]","[0, 0]","[10, 18]","[nan, nan]","[nan, nan]","[nan, nan]",...,"[nan, nan]","[nan, nan]","[nan, nan]","[nan, nan]","[nan, nan]","[nan, nan]","[nan, nan]","[33, 50]","[-17, 17]","[0, 0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24600327,21946,1947-03-29,"[1610610028, 1610610035]","[DEF, HUS]","[Detroit Falcons, Toronto Huskies]","[DEF vs. HUS, HUS @ DEF]","[W, L]","[0, 0]","[24, 25]","[nan, nan]","[nan, nan]","[nan, nan]",...,"[nan, nan]","[nan, nan]","[nan, nan]","[nan, nan]","[nan, nan]","[nan, nan]","[20.0, 21.0]","[66, 63]","[3, -3]","[0, 0]"
24600328,21946,1947-03-29,"[1610612738, 1610610034]","[BOS, BOM]","[Boston Celtics, St. Louis Bombers]","[BOS @ BOM, BOM vs. BOS]","[L, W]","[0, 0]","[24, 23]","[93.0, 90.0]","[0.258, 0.256]","[nan, nan]",...,"[nan, nan]","[nan, nan]","[nan, nan]","[nan, nan]","[nan, nan]","[nan, nan]","[17.0, 13.0]","[55, 59]","[-4, 4]","[0, 0]"
24600329,21946,1947-03-30,"[1610612752, 1610612744]","[NYK, PHW]","[New York Knicks, Philadelphia Warriors]","[NYK @ PHW, PHW vs. NYK]","[L, W]","[0, 0]","[25, 27]","[nan, nan]","[nan, nan]","[nan, nan]",...,"[nan, nan]","[nan, nan]","[nan, nan]","[nan, nan]","[nan, nan]","[nan, nan]","[nan, nan]","[72, 76]","[-4, 4]","[0, 0]"


In [33]:
leaguegamefinder.LeagueGameFinder().get_data_frames()[0].T

,0,1,2,3,4,5,6,7,8,9,...,29990,29991,29992,29993,29994,29995,29996,29997,29998,29999
SEASON_ID,22020,22020,22020,22020,22020,22020,22020,22020,22020,22020,...,22013,22013,22013,22013,22013,22013,22013,22013,22013,22013
TEAM_ID,1610612740,1610612739,1610612745,1610612741,1610612752,1610612756,1610612762,1610612755,1610612760,1610612758,...,1610612744,1610612759,1610612748,1612709910,1612709913,1610612754,1610612756,1612709900,1612709905,1610612747
TEAM_ABBREVIATION,NOP,CLE,HOU,CHI,NYK,PHX,UTA,PHI,OKC,SAC,...,GSW,SAS,MIA,FWN,ERI,IND,PHX,BAK,LAD,LAL
TEAM_NAME,New Orleans Pelicans,Cleveland Cavaliers,Houston Rockets,Chicago Bulls,New York Knicks,Phoenix Suns,Utah Jazz,Philadelphia 76ers,Oklahoma City Thunder,Sacramento Kings,...,Golden State Warriors,San Antonio Spurs,Miami Heat,Fort Wayne Mad Ants,Erie BayHawks,Indiana Pacers,Phoenix Suns,Bakersfield Jam,Los Angeles D-Fenders,Los Angeles Lakers
GAME_ID,0022000067,0022000062,0022000065,0022000063,0022000066,0022000068,0022000068,0022000064,0022000067,0022000065,...,0021301086,0021301084,0021301079,2021300372,2021300371,0021301077,0021301085,2021300375,2021300375,0021301081
GAME_DATE,2020-12-31,2020-12-31,2020-12-31,2020-12-31,2020-12-31,2020-12-31,2020-12-31,2020-12-31,2020-12-31,2020-12-31,...,2014-03-28,2014-03-28,2014-03-28,2014-03-28,2014-03-28,2014-03-28,2014-03-28,2014-03-28,2014-03-28,2014-03-28
MATCHUP,NOP @ OKC,CLE @ IND,HOU vs. SAC,CHI @ WAS,NYK @ TOR,PHX @ UTA,UTA vs. PHX,PHI @ ORL,OKC vs. NOP,SAC @ HOU,...,GSW vs. MEM,SAS @ DEN,MIA @ DET,FWN vs. SPG,ERI @ DEL,IND @ WAS,PHX vs. NYK,BAK vs. LAD,LAD @ BAK,LAL @ MIN
WL,W,L,W,W,L,W,L,W,L,L,...,W,W,W,W,W,L,W,L,W,L
MIN,238,239,240,239,239,240,240,241,239,241,...,240,239,241,241,240,240,239,240,240,241
PTS,113,99,122,133,83,106,95,116,80,119,...,100,133,110,115,158,78,112,117,139,107


In [18]:
leaguegamefinder.LeagueGameFinder(str(team_id)).get_data_frames()[0]

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
game_ids = np.unique(df['GAME_ID'].values)
for game_id in game_ids:
    try:
        dff = boxscoresummaryv2.BoxScoreSummaryV2(str(game_id)).get_data_frames()
        break
    except:
        continue

In [5]:
type(df['GAME_ID'].values[0])

numpy.int64